In [33]:
import urllib.request
import json
import pandas as pd

In [49]:
api_key = "2bb948aa-5b3a-414f-9ce4-3c3b34875f8d"

parameters = ["mean_temp", "mean_wind_speed"]

all_weather_data = []

In [50]:
for param in parameters:
    url = f"https://dmigw.govcloud.dk/v2/climateData/collections/countryValue/items?api-key={api_key}&parameterId={param}&limit=10000&timeResolution=hour"
    
    # Fetch data from API
    with urllib.request.urlopen(url) as fh:
        data = json.load(fh)
    
    # Extract relevant fields
    records = []
    for feature in data.get("features", []):
        properties = feature.get("properties", {})
        records.append({
            "Datetime": properties.get("from"),  # Timestamp
            "Parameter": param,
            "Value": properties.get("value")
        })
    
    df = pd.DataFrame(records)
    all_weather_data.append(df)

final_df = pd.concat(all_weather_data)
final_df = final_df.pivot(index="Datetime", columns="Parameter", values="Value").reset_index()

final_df.to_csv("../data/DMI_data.csv", index=False)